In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
%reload_ext tensorboard

In [ ]:
(train_data, test_data), info = tfds.load("imdb_reviews/subwords8k",
                                          split=(tfds.Split.TRAIN, tfds.Split.TEST),
                                          with_info = True,
                                          as_supervised=True)

encoder = info.features["text"].encoder

train_batches = train_data.shuffle(1000).padded_batch(10,
                                                      padded_shapes=((None,), ()))
test_batches = test_data.shuffle(1000).padded_batch(10,
                                                    padded_shapes=((None, ), ()))

train_batch, train_labels = next(iter(train_batches))

In [ ]:
embedding_dimension = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size,
                                      embedding_dimension)

model = tf.keras.Sequential(
    [
     embedding,
     tf.keras.layers.GlobalAveragePooling1D(),
     tf.keras.layers.Dense(16, activation="relu"),
     tf.keras.layers.Dense(1),
    ]
)

model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [ ]:
log_dir = "./logs/imdb-example"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    for subwords in encoder.subwords:
        f.write(f"{subwords}\n")
    for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
        f.write(f"unknown #{unknown}")

In [ ]:
weights = tf.Variable(model.layers[0].get_weights()[0][1:])
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = "metadata.tsv"
projector.visualize_embeddings(log_dir, config)

In [ ]:
!tensorboard dev upload --logdir ./logs/imdb-example/ \
  --name "Visualizing Word Embeddings from IMDB dataset" \
  --description "Training results from https://colab.research.google.com/drive/1M6-Fv54D-5eaAK0Vp6lHsTN17ud4SG8D#scrollTo=croRpMjisgjj" \
  --one_shot

2021-03-10 06:01:46.355907: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./logs/imdb-example/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%